In [57]:
import os
from dotenv import load_dotenv
from openai import OpenAI
import json
from langchain_community.utilities import GoogleSearchAPIWrapper
from langchain_community.tools import DuckDuckGoSearchRun
from bs4 import BeautifulSoup
from langchain_community.document_loaders.recursive_url_loader import RecursiveUrlLoader
from langchain_community.document_loaders import WebBaseLoader
from langchain import LLMChain, PromptTemplate
from langchain.llms import OpenAI
from langchain.memory import ConversationBufferMemory
from langchain.agents import AgentType, Tool, initialize_agent




from langchain_core.tools import Tool

In [39]:
# Assistent script for web-scraping for real-estate tailored website

def web_scraping_for_website(url):
    
    # Define a custom extractor function to extract text from HTML using BeautifulSoup
    def custom_extractor(html_content):
        soup = BeautifulSoup(html_content, "html.parser")
        return soup.get_text()
    
    # Instantiate the RecursiveUrlLoader
    loader = RecursiveUrlLoader(url=url, extractor=custom_extractor)
    
    # Load the data from the website
    docs = loader.load()
    return docs 
    


In [ ]:

# Load environment variables from .env file
load_dotenv()

# Get the API key from environment variables
openai_api_key = os.getenv("OPENAI_API_KEY")

In [40]:
# declare a LLm
llm = OpenAI(temperature=0.4, openai_api_key=openai_api_key)

In [41]:
city_ranking_template = PromptTemplate(
    input_variables=["country"],
    template="""You are a real estate investment expert. Your task is to identify the top 3 cities for real estate investment in {country}.

Please consider the following factors:
1. Economic growth and stability
2. Real estate market trends
3. Infrastructure development
4. Quality of life
5. Tourism potential
6. Foreign investment policies

Provide a list of up to 3 cities that are the best for real estate investments in {country}. For each city, include:
1. City name
2. Brief explanation (1-2 sentences) of why it's a good investment
3. One key advantage for real estate investment
4. One potential risk or challenge

Format your response as a Python list of dictionaries, like this:
[
    {{
        "name": "City Name",
        "explanation": "Brief explanation",
        "key_advantage": "Key advantage",
        "potential_risk": "Potential risk"
    }},
    # ... (up to 3 cities)
]

Input: Provide the top cities for real estate investment in {country}.
"""
)

# Create LLMChain for city ranking
city_ranking_chain = LLMChain(llm=llm, prompt=city_ranking_template)

# Update the city_ranking_wrapper function
def city_ranking_wrapper(country):
    return city_ranking_chain.run(country=country)


In [42]:

# Neighborhood analysis prompt template
neighborhood_analysis_template = PromptTemplate(
    input_variables=["city"],
    template="""You are a real estate expert. Your task is to identify the top neighborhoods for real estate investment in {city}.

Please provide information on up to 3 neighborhoods, including:
1. Neighborhood name
2. Brief explanation of its appeal for real estate investment
3. Pros (list 2-3)
4. Cons (list 2-3)
5. Average price per square meter (estimate)
6. Population (estimate)
7. Age distribution (general description)
8. Employment rates (general description)
9. Community amenities (list 2-3)
10. Investment rating (1-100000, where higher is better)

Format your response as a Python list of dictionaries, like this:
[
    {{
        "Name": "Neighborhood Name",
        "Explanation": "Brief explanation",
        "Pros": ["Pro 1", "Pro 2"],
        "Cons": ["Con 1", "Con 2"],
        "AveragePricePerSqMeter": "€X,XXX - €X,XXX",
        "Population": "Approx. X,XXX",
        "AgeDistribution": "General description",
        "EmploymentRates": "General description",
        "CommunityAmenities": ["Amenity 1", "Amenity 2"],
        "Ratings": XXXXX
    }},
    # ... (up to 3 neighborhoods)
]

Input: Provide the top neighborhoods for real estate investment in {city}.
"""
)

# Create LLMChain for neighborhood analysis
neighborhood_analysis_chain = LLMChain(llm=llm, prompt=neighborhood_analysis_template)

# Update the neighborhood_analysis_wrapper function
def neighborhood_analysis_wrapper(city):
    return neighborhood_analysis_chain.run(city=city)


In [43]:
# define agent actions 

def define_tools():
    #google_search = GoogleSearchAPIWrapper()
    #duck_duck_search = DuckDuckGoSearchRun()
    super_casa_url = "https://supercasa.pt/en-gb/"
    super_casa_scraping = web_scraping_for_website(super_casa_url)
    tools = [
        Tool(
            name="CityRanking",
            func=city_ranking_wrapper,
            description="Useful for ranking top cities for real estate investment in a given country. Input should be the country name."
        ),
            Tool(
            name="NeighborhoodAnalysis",
            func=neighborhood_analysis_wrapper,
            description="Useful for analyzing the top neighborhoods for real estate investment in a given city. Input should be the city name."
    )
    ]
    return tools 

tools = define_tools()

In [69]:
# configure memory 

memory = ConversationBufferMemory(memory_key="chat_history",input_key='input', output_key="output", return_messages=True)


In [70]:
agent = initialize_agent(tools, 
                                llm, 
                                agent=AgentType.CONVERSATIONAL_REACT_DESCRIPTION, 
                                verbose=True, 
                                memory=memory, 
                                return_intermediate_steps=True)

In [84]:
def run_ai_real_estate_agent():
    print("Welcome to the AI Real Estate Agent!")
    print("I can help you find the best cities and neighborhoods for real estate investment in a specific country.")
    
    country = input("Please enter the name of the country you're interested in: ")

    #try:
    # Get top cities
    cities_result = agent(f"What are the top 3 cities for real estate investment in {country}?")
    print(cities_result["intermediate_steps"])
    intermediate_steps_cities = cities_result["intermediate_steps"] # Parse JSON response directly
    if intermediate_steps_cities:
        cities_output = intermediate_steps_cities[0][1]
        if cities_output.strip():
            cities = json.loads(cities_output)

    # Analyze neighborhoods for each city
    country_data = {
        "Country": {
            "Name": country,
            "Cities": []
        }
    }

    for city in cities:
        neighborhoods_result = agent(f"Analyze the top 7 neighborhoods for real estate investment in {city['name']}.")
        intermediate_steps_neighobhoods = neighborhoods_result["intermediate_steps"] # Convert string representation of list to actual list
        if intermediate_steps_cities:
            neighborhoods_output = intermediate_steps_neighobhoods[0][1]
            if neighborhoods_output.strip():
                neighborhoods = json.loads(neighborhoods_output)

        city_data = {
            "Name": city['name'],
            "Neighborhoods": neighborhoods
        }
        country_data["Country"]["Cities"].append(city_data)

    # Generate final JSON output
    print("\nAI Real Estate Agent Report:")
    print(json.dumps(country_data, indent=2))

    #except Exception as e:
    print(f"An error occurred: {str(e)}")


In [85]:
if __name__ == "__main__":
    run_ai_real_estate_agent()

Welcome to the AI Real Estate Agent!
I can help you find the best cities and neighborhoods for real estate investment in a specific country.


Please enter the name of the country you're interested in:  israel




> Entering new AgentExecutor chain...
Thought: Do I need to use a tool? Yes
Action: CityRanking
Action Input: Israel
Observation: 
[
    {
        "name": "Tel Aviv",
        "explanation": "Tel Aviv is the economic and cultural center of Israel, with a strong and stable economy. It also has a thriving real estate market and a high quality of life.",
        "key_advantage": "Strong economic growth and stability",
        "potential_risk": "High property prices and competition for investment opportunities"
    },
    {
        "name": "Jerusalem",
        "explanation": "Jerusalem is a major religious and cultural hub, attracting a large number of tourists and foreign investors. It also has a growing tech sector and a well-developed infrastructure.",
        "key_advantage": "Tourism potential and foreign investment policies",
        "potential_risk": "Political tensions and potential for security risks"
    },
    {
        "name": "Haifa",
        "explanation": "Haifa is a major 

JSONDecodeError: Unterminated string starting at: line 17 column 24 (char 935)